# Sports Statistics

Below you'll find statistics for the following sports:
- Ice Hockey
- Athletics
- Shooting
- Cross Country Skiing

(The four sports that Sweden got the most medals in)

In [21]:
import pandas as pd
import plotly_express as px

In [22]:
# Reading the athlete_events.csv file
athlete_events = pd.read_csv("data/athlete_events.csv")

In [23]:
# NOTE: Create a new DataFrame instance named df_icehockey using pandas.
# Filter the rows from the existing DataFrame athlete_events based on the condition 'Sport == "XX"'.
# The resulting DataFrame, df_icehockey, contains data specific to athletes and events in the sport "XX".
df_icehockey = pd.DataFrame(athlete_events.query('Sport == "Ice Hockey"'))
df_athletics = pd.DataFrame(athlete_events.query('Sport == "Athletics"'))
df_shooting = pd.DataFrame(athlete_events.query('Sport == "Shooting"'))
df_cross_county_skiing = pd.DataFrame(athlete_events.query('Sport == "Cross Country Skiing"'))

selected_sports = ['Ice Hockey', 'Athletics', 'Shooting', 'Cross Country Skiing']
df_all_selected_sports = pd.DataFrame(athlete_events.query('Sport in @selected_sports'))

# NOTE: Displaying the length of the df:s using display method
display(f"{len(df_icehockey)=}")
display(f"{len(df_athletics)=}")
display(f"{len(df_shooting)=}")
display(f"{len(df_cross_county_skiing)=}")
display(f"{len(df_all_selected_sports)=}")

'len(df_icehockey)=5516'

'len(df_athletics)=38624'

'len(df_shooting)=11448'

'len(df_cross_county_skiing)=9133'

'len(df_all_selected_sports)=64721'

In [24]:
df_all_selected_sports.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
10,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 10 kilometres,NaN
11,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 50 kilometres,NaN
12,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 10/15 kilometres Pu...,NaN
13,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 4 x 10 kilometres R...,NaN
14,6,Per Knut Aaland,M,33.0,188.0,75.0,United States,USA,1994 Winter,1994,Winter,Lillehammer,Cross Country Skiing,Cross Country Skiing Men's 10 kilometres,NaN


#### Medaljfördelning mellan länder i sporterna

In [25]:
medal_distribution = (
    df_all_selected_sports.groupby(['NOC', 'Sport', 'Medal'])
    .size()
    .reset_index(name="Number of Medals")
)

# Plotting a histogram of ages of athletes from SWE in all olympic games around the mean age
fig = px.histogram(
    medal_distribution, 
    x="NOC",
    y="Number of Medals",
    log_y=True,
    color='Sport',
    title="Medal Distribution in All Olympic Games",
    barmode='group',
)

# Making spaces between bars
fig.update_layout(bargap=0.01)

fig.show()

In [26]:
px.sunburst(medal_distribution, values='Number of Medals', path=['Sport', 'NOC'])

#### Åldersfördelning i sporterna

In [27]:
mean_age_all = round(df_all_selected_sports["Age"].mean(), 1)

age_distribution = (
    df_all_selected_sports.groupby(['Sport', 'Age'])
    .size()
    .reset_index(name="Number of Athletes")
)

# Plotting a histogram of ages of athletes from SWE in all olympic games around the mean age
fig = px.histogram(
    age_distribution, 
    x="Age",
    y="Number of Athletes",
    log_y=True,
    color='Sport',
    title="Age distribution in All Olympic Games",
    range_x=[mean_age_all-20, mean_age_all+20],
    barmode='group',
    nbins=80
)

# Mark the mean age in the x-axis
fig.add_vline(x=mean_age_all, line_width=3, line_dash="dash", line_color="green", annotation_text="Mean age all sports")

# Making spaces between bars
fig.update_layout(bargap=0.01)

fig.show()

In [28]:
px.sunburst(age_distribution, values='Number of Athletes', path=['Sport', 'Age'])

#### Skapa fler plots för att visualisera olika aspekter kring sporterna.

#### FOR DASHBOARD

#### Dataframe for countries, contintents and coordinates

#### Countries

In [29]:
# Reading the noc_regions.csv file
noc_regions = pd.read_csv("data/noc_regions.csv")
noc_regions.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [30]:
# Renaming the column "region" to "Country" in the noc_regions DataFrame
noc_regions.rename(columns={"region": "Country"}, inplace=True)

#### Coordinates

In [31]:
# Read countries and their coordinates from the csv file
country_coordinates = pd.read_csv("data/country_coordinates.csv")

# Rename the columns
country_coordinates.rename(columns={"country": "Country", "latitude":"Country_latitude", "longitude":"Country_longitude"}, inplace=True)

country_coordinates.head()

,country_code,Country_latitude,Country_longitude,Country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas
3,AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona
4,AI,18.220554,-63.068615,Anguilla,CA,36.778261,-119.417932,California


In [32]:
# Drop all columns except latitude, longitude and Country
country_coordinates.drop(['country_code', 'usa_state_code', 'usa_state_latitude', 'usa_state_longitude', 'usa_state'], axis=1, inplace=True)

#### Merging coordinates

In [33]:
 # Add to noc_regions DataFrame the coordinates of each country
noc_regions = pd.merge(noc_regions, country_coordinates, left_on='Country', right_on='Country')

In [34]:
noc_regions.head()

,NOC,Country,notes,Country_latitude,Country_longitude
0,AFG,Afghanistan,NaN,33.939110,67.709953
1,ALB,Albania,NaN,41.153332,20.168331
2,ALG,Algeria,NaN,28.033886,1.659626
3,AND,Andorra,NaN,42.546245,1.601554
4,ANG,Angola,NaN,-11.202692,17.873887


In [35]:
# Show entire dataframe
pd.set_option('display.max_rows', None)

In [36]:
noc_regions["NOC"].unique()

array(['AFG', 'ALB', 'ALG', 'AND', 'ANG', 'ANZ', 'AUS', 'ARG', 'ARM',
       'ARU', 'ASA', 'AUT', 'AZE', 'BAH', 'BAN', 'BAR', 'BDI', 'BEL',
       'BEN', 'BER', 'BHU', 'BIH', 'BIZ', 'BLR', 'BOH', 'CZE', 'TCH',
       'BOT', 'BRA', 'BRN', 'BRU', 'BUL', 'BUR', 'CAF', 'CAM', 'CAN',
       'NFL', 'CAY', 'CHA', 'CHI', 'CHN', 'HKG', 'CMR', 'COK', 'COL',
       'COM', 'CPV', 'CRC', 'CRO', 'CRT', 'GRE', 'CUB', 'CYP', 'DEN',
       'DJI', 'DMA', 'DOM', 'ECU', 'EGY', 'ERI', 'ESA', 'ESP', 'EST',
       'ETH', 'EUN', 'RUS', 'URS', 'FIJ', 'FIN', 'FRA', 'FRG', 'GDR',
       'GER', 'SAA', 'FSM', 'GAB', 'GAM', 'GBS', 'GEO', 'GEQ', 'GHA',
       'GRN', 'GUA', 'GUI', 'GUM', 'GUY', 'HAI', 'HON', 'HUN', 'INA',
       'IND', 'IRI', 'IRL', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR',
       'JPN', 'KAZ', 'KEN', 'KGZ', 'KIR', 'KOR', 'KOS', 'KSA', 'KUW',
       'LAO', 'LAT', 'LBA', 'LBR', 'LCA', 'LES', 'LIB', 'LIE', 'LTU',
       'LUX', 'MAD', 'MAL', 'MAS', 'NBO', 'MAR', 'MAW', 'MDA', 'MDV',
       'MEX', 'MGL',

#### Continents

In [37]:
import pycountry_convert as pc
import pycountry

# Creating a DataFrame with 'Country' and 'Continent' columns, excluding special cases               # Källa: https://stackoverflow.com/questions/55910004/how-to-get-continent-name-from-country-using-pycountry
def get_continent(country):
    try:
        return pc.country_alpha2_to_continent_code(country.alpha_2)
    except (KeyError, AttributeError):
        return 'Unknown'
    
country_continent = pd.DataFrame([
    {'Country': country.name, 'Continent': get_continent(country)}
    for country in pycountry.countries
])

In [38]:
country_continent.head()

,Country,Continent
0,Aruba,NA
1,Afghanistan,AS
2,Angola,AF
3,Anguilla,NA
4,Åland Islands,EU


In [39]:
country_continent.info()
# Kontroll, hittills finns alla länder med

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    249 non-null    object
 1   Continent  249 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [40]:
country_continent.query('Country == "United States"')

,Country,Continent
234,United States,NA


In [41]:
# Find all values in Country column of country continent that do not have a match in noc_regions
noc_regions[~noc_regions['Country'].isin(country_continent['Country'])]

,NOC,Country,notes,Country_latitude,Country_longitude
24,BOH,Czech Republic,Bohemia,49.817492,15.472962
25,CZE,Czech Republic,NaN,49.817492,15.472962
26,TCH,Czech Republic,NaN,49.817492,15.472962
30,BRU,Brunei,NaN,4.535277,114.727669
46,CPV,Cape Verde,NaN,16.002082,-24.013197
64,EUN,Russia,NaN,61.524010,105.318756
65,RUS,Russia,NaN,61.524010,105.318756
66,URS,Russia,NaN,61.524010,105.318756
74,FSM,Micronesia,NaN,7.425554,150.550812
91,IRI,Iran,NaN,32.427908,53.688046


In [42]:
# Find all values in Country column of country continent that do not have a match in noc_regions
country_continent[~country_continent['Country'].isin(noc_regions['Country'])]


,Country,Continent
3,Anguilla,NA
4,Åland Islands,EU
11,Antarctica,Unknown
12,French Southern Territories,Unknown
13,Antigua and Barbuda,NA
20,"Bonaire, Sint Eustatius and Saba",NA
27,Saint Barthélemy,NA
31,"Bolivia, Plurinational State of",SA
34,Brunei Darussalam,AS
36,Bouvet Island,AN


In [43]:
dict_respell = {"Czechia": "Czech Republic", "Brunei Darussalam": "Brunei", 
"Cabo Verde": "Cape Verde", "Iran, Islamic Republic of": "Iran", "Korea, Republic of": "South Korea", 
"Korea, Democratic People's Republic of": "North Korea", "Syrian Arab Republic":"Syria",  
"Russia": "Russian Federation", "Taiwan": "Taiwan, Province of China", 
"Vietnam": "Viet Nam", "Venezuela, Bolivarian Republic of": "Venezuela", "United States": "USA"}

In [44]:
# Replace the values in the Country column of country_continent with the values in the dict_respell dictionary
country_continent["Country"].replace(dict_respell, inplace=True)

# Kontroll, hittills ok.

In [45]:
country_continent.query('Country == "USA"')

# Kontroll, hittills ok.

,Country,Continent
234,USA,NA


In [46]:
# Create dataframe mapping continents to abbreviations. 
continents_mapped = {
    'AF': 'Africa',
    'AN': 'Antarctica',
    'AS': 'Asia',
    'EU': 'Europe',
    'NA': 'North America',
    'OC': 'Oceania',
    'SA': 'South America',
    'Unknown': 'Unknown'
}

In [47]:
# Change names in final_df Continent column to match the continent_mapped in the contry_continent DataFrame
country_continent['Continent'] = country_continent['Continent'].map(continents_mapped)
country_continent.head()

,Country,Continent
0,Aruba,North America
1,Afghanistan,Asia
2,Angola,Africa
3,Anguilla,North America
4,Åland Islands,Europe


In [48]:
country_continent.query('Country == "USA"')
# Kontroll, fortfarande ok

,Country,Continent
234,USA,North America


In [50]:
# Creating a DataFrame with 'Continent', 'Latitude' and 'Longitude' columns
continent_coordinates = pd.DataFrame({
    'Continent': ['Africa', 'Antarctica', 'Asia', 'Europe', 'North America', 'Oceania', 'South America'],
    'Continent_latitude': [7.1881, -82.8628, 34.0479, 54.5260, 54.5260, -14.2350, -14.2350],
    'Continent_longitude': [21.0938, 135, 100.6197, 15.2551, -105.2551, 141.2551, -59.2551]
})


In [51]:
# Adding the continent coordinates 
country_continent = pd.merge(country_continent, continent_coordinates, left_on="Continent", right_on="Continent")

In [52]:
country_continent.query("Country == 'USA'")

# Kontroll, fortfarande OK

,Country,Continent,Continent_latitude,Continent_longitude
36,USA,North America,54.526,-105.2551


#### Merging 

In [54]:
# HÄR HAR DEN FÖRSVUNNIT, VARFÖR??????

,NOC,Country,notes,Country_latitude,Country_longitude


In [41]:
# Merging the Dataframes athlete_events and noc_regions on the column "NOC"
noc_regions = pd.merge(noc_regions, country_continent, left_on="Country", right_on="Country")

In [42]:
noc_regions.head()

,NOC,Country,notes,Country_latitude,Country_longitude,Continent,Continent_latitude,Continent_longitude
0,AFG,Afghanistan,NaN,33.939110,67.709953,Asia,34.0479,100.6197
1,ALB,Albania,NaN,41.153332,20.168331,Europe,54.5260,15.2551
2,ALG,Algeria,NaN,28.033886,1.659626,Africa,7.1881,21.0938
3,AND,Andorra,NaN,42.546245,1.601554,Europe,54.5260,15.2551
4,ANG,Angola,NaN,-11.202692,17.873887,Africa,7.1881,21.0938


In [45]:
noc_regions.query('Country == "USA"')


,NOC,Country,notes,Country_latitude,Country_longitude,Continent,Continent_latitude,Continent_longitude


#### Dataframe for participants and medals per year

In [ ]:
# Dataframe for participants and medals per year
grouped_participants_medals= final_df.groupby(['Year', 'Season', 'Country', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})


In [ ]:
grouped_participants_medals.head()

In [ ]:
# Using plotly_express and mapbox to plot the amount count of Name per year and country

fig = px.scatter_mapbox(grouped_participants_medals, lat="Country_latitude", lon="Country_longitude", size="Name", color="Medal", height = 800, width= 900 ,hover_name = "Country")

display(fig.update_layout(mapbox_style="open-street-map", title = "Size according to count of participants="))


#### Dataframe for sports per year and country

In [ ]:
# Dataframe for sports per year and country
grouped_sports = final_df.groupby(['Year', 'Season', 'Country','Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False).agg(
    {'Sport': 'nunique'})




In [ ]:
grouped_sports.head()

#### Grouped gender data per country

In [ ]:
# Grouped per country and gender
grouped_gender_country = final_df.groupby(['Year', 'Sex', 'Country', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False).agg(
    {'Name': 'nunique'})

# Rename the column "Name" to "Count"
grouped_gender_country.rename(columns={"Name": "Count"}, inplace=True)


In [ ]:
grouped_gender_country.head()

In [ ]:
# Create a column named ratio for grouped_gender_country dataframe
# In this column, calculate the ratio of M and F athletes per country and year

grouped_gender_country['Ratio'] = grouped_gender_country.groupby(['Year', 'Country'])['Count'].transform(lambda x: x / x.sum())    # Källa


#### Grouped gender data per continent

In [ ]:
# Grouped per continent and gender
grouped_gender_continent = final_df.groupby(['Year', 'Sex', 'Continent','Continent_latitude', 'Continent_longitude'], as_index=False).agg(
{'Name': 'nunique'})


# Rename the column "Name" to "Count"

grouped_gender_continent.rename(columns={"Name": "Count"}, inplace=True)


In [ ]:
# Create a column named ratio for grouped_gender_continent dataframe
# In this column, calculate the ratio of M and F athletes per continent and year

grouped_gender_continent['Ratio'] = grouped_gender_continent.groupby(['Year', 'Continent'])['Count'].transform(lambda x: x / x.sum())

In [ ]:
grouped_gender_continent.head()

#### Grouped gender data total for all countries 

In [ ]:
# Grouped data for alla countries and gender
grouped_gender_total = final_df.groupby(['Year', 'Sex'], as_index=False).agg(
{'Name': 'nunique'})

# Rename the column "Name" to "Count"
grouped_gender_total.rename(columns={"Name": "Count"}, inplace=True)

In [ ]:
# Create a column named ratio for grouped_gender_total dataframe
# In this column, calculate the ratio of M and F athletes per continent and year

grouped_gender_total['Ratio'] = grouped_gender_total.groupby(['Year'])['Count'].transform(lambda x: x / x.sum())

In [ ]:
grouped_gender_total.head()